In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import joblib
import os
import sys
import logging

# Configurar logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Agregar la ruta raíz para importar módulos
# En Jupyter, usamos getcwd() en lugar de __file__
notebook_dir = os.getcwd()
project_root = os.path.dirname(notebook_dir)
sys.path.insert(0, project_root)

from config import SCALER_PATH, RISK_INVERSE_MAPPING, FEATURE_NAMES
from mlflow_manager import mlflow_manager

print("="*60)
print("🧪 PRUEBAS DE INFERENCIA - SALUD MATERNAL")
print("="*60)

# --- 1. CARGAR MODELO Y SCALER ---
print("\n🔄 Cargando recursos...")
try:
    scaler = joblib.load(str(SCALER_PATH))
    print("✅ Scaler cargado")
except Exception as e:
    logger.error(f"❌ Error cargando scaler: {e}")

try:
    model = mlflow_manager.load_model()
    print("✅ Modelo cargado desde MLflow")
except Exception as e:
    logger.error(f"❌ Error cargando modelo: {e}")
    raise

# --- 2. FUNCIÓN DE PREDICCIÓN ---
def predict_risk(age, sysbp, diabp, bs, bodytemp, heartrate):
    """Realiza predicción de riesgo materno"""
    
    # Crear array de entrada
    input_data = np.array([[age, sysbp, diabp, bs, bodytemp, heartrate]])
    
    # Escalar
    scaled_data = scaler.transform(input_data)
    
    # Predicción
    prediction = model.predict(scaled_data, verbose=0)
    class_idx = np.argmax(prediction)
    confidence = float(prediction[0][class_idx]) * 100
    
    result = {
        "features": dict(zip(FEATURE_NAMES, input_data[0])),
        "prediction": RISK_INVERSE_MAPPING[class_idx],
        "confidence": confidence,
        "class_id": class_idx
    }
    
    return result

# --- 3. CASOS DE PRUEBA ---
print("\n🧪 Ejecutando casos de prueba...\n")

test_cases = [
    {
        "name": "Paciente Bajo Riesgo",
        "data": [25, 110, 70, 6.5, 98.0, 72]
    },
    {
        "name": "Paciente Riesgo Medio",
        "data": [35, 135, 85, 10.0, 99.5, 80]
    },
    {
        "name": "Paciente Alto Riesgo",
        "data": [42, 160, 100, 14.0, 101.5, 95]
    }
]

for test in test_cases:
    print(f"📋 {test['name']}")
    print(f"   Edad={test['data'][0]}, SBP={test['data'][1]}, DBP={test['data'][2]}, " +
          f"BS={test['data'][3]}, Temp={test['data'][4]}, HR={test['data'][5]}")
    
    result = predict_risk(*test['data'])
    print(f"   ✅ Predicción: {result['prediction']}")
    print(f"   📊 Confianza: {result['confidence']:.2f}%")
    print()

# --- 4. TRANSFER LEARNING / CONTINUOUS LEARNING ---
print("\n🔄 TRANSFER LEARNING - Ajuste con nuevos datos")
print("-"*60)

def retrain_with_new_data(new_samples_X, new_samples_y):
    """Realiza transfer learning con nuevos datos"""
    
    print(f"📊 Nuevas muestras: {len(new_samples_X)}")
    
    with mlflow_manager.start_run(run_name="Continuous_Learning_Update"):
        # Transfer Learning: congelar primeras capas
        print("🔒 Congelando primeras capas...")
        for i, layer in enumerate(model.layers[:-1]):
            layer.trainable = False
            print(f"   - {layer.name}: frozen")
        
        # Recompilar con learning rate más bajo
        print("⚙️ Recompilando modelo...")
        model.compile(
            optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
            loss='sparse_categorical_crossentropy',
            metrics=['accuracy']
        )
        
        # Entrenar con nuevos datos
        print("🔄 Entrenando con nuevos datos...")
        history = model.fit(
            new_samples_X, 
            new_samples_y,
            epochs=10,
            verbose=1,
            batch_size=2
        )
        
        # Guardar modelo actualizado
        print("💾 Guardando modelo actualizado...")
        mlflow_manager.log_model(model, "fine_tuned_model")
        print("✅ Modelo re-entrenado y guardado en MLflow")
        
        return history

# Crear datos simulados para re-entrenamiento
print("\n📝 Simulando nuevos pacientes para aprendizaje continuo...")
new_X = np.array([
    [28, 125, 80, 8.0, 98.5, 75],   # Riesgo bajo
    [38, 145, 90, 11.5, 100.0, 85], # Riesgo medio
])
new_y = np.array([0, 1])

# Escalar nuevos datos
new_X_scaled = scaler.transform(new_X)

# Ejecutar re-entrenamiento
print("\n🔄 Iniciando re-entrenamiento...\n")
history = retrain_with_new_data(new_X_scaled, new_y)

print("\n" + "="*60)
print("✅ PRUEBAS COMPLETADAS")
print("="*60)
print(f"📍 Revisar MLflow para ver el run actualizado")
print(f"📍 Run ID más reciente: {mlflow_manager.get_run_id()}")

INFO:mlflow_manager:✅ MLflow URI configurada: sqlite:///c:\Users\andre\OneDrive\Desktop\Proyecto\mlflow.db
2026/02/01 18:37:36 INFO alembic.runtime.plugins: setup plugin alembic.autogenerate.schemas
2026/02/01 18:37:36 INFO alembic.runtime.plugins: setup plugin alembic.autogenerate.tables
2026/02/01 18:37:36 INFO alembic.runtime.plugins: setup plugin alembic.autogenerate.types
2026/02/01 18:37:36 INFO alembic.runtime.plugins: setup plugin alembic.autogenerate.constraints
2026/02/01 18:37:36 INFO alembic.runtime.plugins: setup plugin alembic.autogenerate.defaults
2026/02/01 18:37:36 INFO alembic.runtime.plugins: setup plugin alembic.autogenerate.comments
2026/02/01 18:37:37 INFO alembic.runtime.migration: Context impl SQLiteImpl.
2026/02/01 18:37:37 INFO alembic.runtime.migration: Will assume non-transactional DDL.
INFO:mlflow_manager:✅ Experimento existente: Maternal_Health_Project


🧪 PRUEBAS DE INFERENCIA - SALUD MATERNAL

🔄 Cargando recursos...
✅ Scaler cargado


INFO:mlflow_manager:✅ Modelo cargado exitosamente desde run: 90caeeb84fcf4186899ba211e515b7ae
c:\Users\andre\OneDrive\Desktop\Proyecto\venv2\Lib\site-packages\sklearn\utils\validation.py:2691: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\andre\OneDrive\Desktop\Proyecto\venv2\Lib\site-packages\sklearn\utils\validation.py:2691: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\andre\OneDrive\Desktop\Proyecto\venv2\Lib\site-packages\sklearn\utils\validation.py:2691: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


✅ Modelo cargado desde MLflow

🧪 Ejecutando casos de prueba...

📋 Paciente Bajo Riesgo
   Edad=25, SBP=110, DBP=70, BS=6.5, Temp=98.0, HR=72
   ✅ Predicción: Bajo Riesgo (Low)
   📊 Confianza: 57.81%

📋 Paciente Riesgo Medio
   Edad=35, SBP=135, DBP=85, BS=10.0, Temp=99.5, HR=80
   ✅ Predicción: Alto Riesgo (High)
   📊 Confianza: 86.32%

📋 Paciente Alto Riesgo
   Edad=42, SBP=160, DBP=100, BS=14.0, Temp=101.5, HR=95
   ✅ Predicción: Alto Riesgo (High)
   📊 Confianza: 99.40%


🔄 TRANSFER LEARNING - Ajuste con nuevos datos
------------------------------------------------------------

📝 Simulando nuevos pacientes para aprendizaje continuo...

🔄 Iniciando re-entrenamiento...

📊 Nuevas muestras: 2
🔒 Congelando primeras capas...
   - base_layer_1: frozen
   - base_layer_2: frozen
   - dropout: frozen
⚙️ Recompilando modelo...
🔄 Entrenando con nuevos datos...
Epoch 1/10


c:\Users\andre\OneDrive\Desktop\Proyecto\venv2\Lib\site-packages\sklearn\utils\validation.py:2691: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 415ms/step - accuracy: 0.0000e+00 - loss: 3.1686
Epoch 2/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - accuracy: 0.5000 - loss: 2.3818
Epoch 3/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - accuracy: 0.5000 - loss: 2.3512
Epoch 4/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - accuracy: 0.0000e+00 - loss: 3.1356
Epoch 5/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - accuracy: 0.0000e+00 - loss: 3.2841
Epoch 6/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - accuracy: 0.0000e+00 - loss: 3.1333
Epoch 7/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - accuracy: 0.0000e+00 - loss: 3.1617
Epoch 8/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - accuracy: 0.0000e+00 - loss: 2.2113
Epoch 9/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - accuracy: 0.0000e+00 - loss: 2.6960
Epoch 10/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - accuracy: 0.0000e+00 - loss: 3.1287


2026/02/01 18:37:38 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


💾 Guardando modelo actualizado...


2026/02/01 18:37:38 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.
INFO:mlflow_manager:✅ Modelo registrado en MLflow: fine_tuned_model


✅ Modelo re-entrenado y guardado en MLflow

✅ PRUEBAS COMPLETADAS
📍 Revisar MLflow para ver el run actualizado
📍 Run ID más reciente: 20d2f970af6e4ec49510c49004d01c19
